In [2]:
# 1. Instalace potřebných knihoven
!pip install -q transformers[torch] datasets evaluate accelerate huggingface_hub

import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from huggingface_hub import notebook_login
import numpy as np
import evaluate

# 2. Přihlášení k Hugging Face
# Po spuštění vlož svůj token s právy "Write"
notebook_login()

# 3. Načtení datasetu a příprava labelů
dataset = load_dataset("marekk/testing_dataset_article_category", split="train")
dataset = dataset.train_test_split(test_size=0.2)

# Získání názvů tříd a jejich počtu
labels = dataset["train"].features["label"].names
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
num_labels = len(labels)

# 4. Tokenizace (použijeme XLM-RoBERTa pro 30 jazyků)
model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def preprocess_function(examples):
    # Texty zkrátíme na 256 tokenů pro rychlejší trénink v Colabu
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# 5. Definice metriky pro vyhodnocení
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 6. Načtení modelu
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

# 7. Konfigurace tréninku
# Změň 'moje-uzivatelske-jmeno' na své skutečné jméno na HF
repo_name = "article-classifier-multilingual"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True, # Tímto se model nahraje na tvůj profil
    report_to="none"
)

# 8. Trénink
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"] if "test" in tokenized_dataset else tokenized_dataset["train"].select(range(100)), # fallback pokud není test split
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

# 9. Finální nahrání na Hugging Face Hub
trainer.push_to_hub("Trénink dokončen!")

KeyboardInterrupt: 